In [ ]:
import os
import polars as pl
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from src.model import GestureBranchedModel
from src.dataset import *
from pathlib import Path
import yaml
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Load model and config
exp_dir = Path('../experiments/cmi_training_20250814_220701')
with open(exp_dir / 'configs' / 'config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Initialize model
model = GestureBranchedModel(
    num_classes=config['model']['num_classes'],
    d_model=config['model']['d_model'],
    d_reduced=config['model']['d_reduced'],
    num_heads=config['model']['num_heads'],
    num_layers=config['model']['num_layers'],
    dropout=config['model']['dropout'],
    max_seq_length=config['model']['max_seq_length'],
    sequence_processor=config['model']['sequence_processor'],
    tof_backbone=config['model']['tof_backbone']
)

# Load trained weights
checkpoint = torch.load(exp_dir / 'models/best_model.pt', map_location='cpu')
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)
model.eval()

print("Model loaded successfully")

In [ ]:
# Load training data for label encoding setup
dataset_dir = Path('../dataset')
train_sequences_df = pl.read_csv(dataset_dir / 'train.csv')
train_sequences_df = train_sequences_df.fill_null(-1.0).fill_nan(-1.0)

# Prepare gesture labels and get label encoder
train_sequences_df, labelencoder, target_gesture_id, non_target_gesture_id = prepare_gesture_labels(train_sequences_df)

# Get gesture classes for mapping predictions back to gesture names
gesture_classes = list(labelencoder.classes_)
print(f"Loaded {len(gesture_classes)} gesture classes")

In [ ]:
def predict(data_batch):
    """
    Predict gesture for a single sequence.
    
    Args:
        data_batch: Tuple of (sequence_df, demographics_df) from the inference server
        
    Returns:
        str: Predicted gesture name
    """
    sequence_df, demographics_df = data_batch
    
    # Process sequence into chunks
    sequence_processor = SequenceProcessor()
    sequences = sequence_processor.process_dataframe(
        df=sequence_df, 
        max_seq_length=config['data']['max_seq_length']
    )
    
    if not sequences:
        return "Text on phone"  # Default prediction if no valid sequences
    
    # Create dataset for this sequence
    dataset = CMIDataset(sequences=sequences, max_length=config['data']['max_seq_length'])
    
    # Collect all chunks for this sequence
    batch_data = []
    for data in dataset:
        batch_data.append(data)
    
    if not batch_data:
        return "Text on phone"  # Default prediction
    
    # Stack tensors for batch processing
    tof_batch = torch.stack([d['tof'] for d in batch_data]).to(device)  # (batch_size, seq_len, 320)
    acc_batch = torch.stack([d['acc'] for d in batch_data]).to(device)  # (batch_size, seq_len, 3)
    rot_batch = torch.stack([d['rot'] for d in batch_data]).to(device)  # (batch_size, seq_len, 4)
    thm_batch = torch.stack([d['thm'] for d in batch_data]).to(device)  # (batch_size, seq_len, 5)
    chunk_start_idx_batch = torch.stack([d['chunk_start_idx'] for d in batch_data]).to(device)
    
    # Forward pass
    with torch.no_grad():
        logits = model(
            tof_features=tof_batch,
            acc_features=acc_batch,
            rot_features=rot_batch,
            thm_features=thm_batch,
            chunk_start_idx=chunk_start_idx_batch
        )  # (batch_size, num_classes)
    
    # Average predictions across all chunks and get predicted class
    mean_logits = logits.mean(dim=0)  # (num_classes,)
    predicted_class_id = mean_logits.argmax().item()
    
    # Map class ID back to gesture name
    predicted_gesture = gesture_classes[predicted_class_id]
    
    return predicted_gesture

print("Predict function defined")

In [ ]:
# Test the predict function with sample data
test_sequences_df = pl.read_csv(dataset_dir / 'test.csv')
test_sequences_df = test_sequences_df.fill_null(-1.0).fill_nan(-1.0)
test_demographics_df = pl.read_csv(dataset_dir / 'test_demographics.csv')

# Get a sample sequence for testing
sample_sequence_id = test_sequences_df['sequence_id'][0]
sample_sequence = test_sequences_df.filter(pl.col('sequence_id') == sample_sequence_id)
sample_demographics = test_demographics_df.filter(pl.col('subject') == sample_sequence['subject'][0])

# Test prediction
test_prediction = predict((sample_sequence, sample_demographics))
print(f"Sample prediction for sequence {sample_sequence_id}: {test_prediction}")

In [ ]:
# Initialize and run the inference server
import kaggle_evaluation.cmi_inference_server

inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    # Running in Kaggle competition environment
    inference_server.serve()
else:
    # Running locally for testing
    inference_server.run_local_gateway(
        data_paths=(
            '/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv',
            '/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv',
        )
    )

# Show results if running locally
if not os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    try:
        results = pd.read_parquet("submission.parquet")
        print("Submission results:")
        print(results.head())
        print(f"Total predictions: {len(results)}")
    except FileNotFoundError:
        print("No submission.parquet file found - may need to run with proper data paths")